# Proposed Model VS Speckle2Void, SAR2SAR and CNN-NLM

In [1]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from DatasetHandler import DatasetHandler
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
import numpy as np

In [ ]:
def plot_model_results(batch_speckle, batch_clean, batch_pred):
    n = batch_speckle.shape[0]

    for i in range(n):
        fig, axes = plt.subplots(
        nrows = 2,
        ncols = 4, 
        figsize = (16, 8))

        print('-------------------------------------------------------------------------------------------')
        print('Test', i)

        axes[0,0].imshow(batch_speckle[i,...,0], cmap='gray')
        axes[0,0].set_title('Input with speckle')
        axes[0,1].imshow(batch_clean[i,...,0], cmap='gray')
        axes[0,1].set_title('Ground truth')
        axes[0,2].imshow(batch_pred[i,...,0], cmap='gray')
        axes[0,2].set_title('Model Prediction')
        diff = np.abs(batch_pred[i,...,0] - batch_clean[i,...,0])
        axes[0,3].imshow(diff, vmin=np.min(diff), vmax=np.max(diff), cmap='gray')
        axes[0,3].set_title('|Model Prediction - Ground Truth|')
        
        try:
            axes[1,0].hist(batch_speckle[i,...,0].flatten(), bins=20, histtype='step')
            axes[1,0].set_title('Input with speckle')
            axes[1,1].hist(batch_clean[i,...,0].flatten(), bins=20, histtype='step')
            axes[1,1].set_title('Ground truth')
            axes[1,2].hist(batch_pred[i,...,0].flatten(), bins=20, histtype='step')
            axes[1,2].set_title('Model Prediction')
            axes[1,3].hist(diff.flatten(), bins=20, histtype='step')
            axes[1,3].set_title('|Model Prediction - Ground Truth|')
        except:
            pass
    
        plt.show()
        plt.close()

def compute_metrics(batch_speckle, batch_clean, batch_pred):
    n = batch_speckle.shape[0]
    
    print('===========================================================================================================================================')
    print('  Test \t\t Metric\t\tGrount Truth VS Grount Truth \t\t Grount Truth VS Input \t\t Grount Truth VS Model Prediction')
    print('-------------------------------------------------------------------------------------------------------------------------------------------')
    for i in range(n):
        gt_vs_gt = peak_signal_noise_ratio(batch_clean[i, ...,0], batch_clean[i, ...,0], data_range=1.0)
        gt_vs_in  = peak_signal_noise_ratio(batch_clean[i, ...,0], batch_speckle[i,...,0], data_range=1.0)
        gt_vs_pred  = peak_signal_noise_ratio(batch_clean[i, ...,0], batch_pred[i,...,0], data_range=1.0)

        print('   %i  \t\t  PSNR \t\t             %.2f             \t\t             %.2f      \t\t             %.2f' % (i, gt_vs_gt, gt_vs_in, gt_vs_pred))
        
        gt_vs_gt = structural_similarity(batch_clean[i, ...,0], batch_clean[i, ...,0], data_range=1.0)
        gt_vs_in  = structural_similarity(batch_clean[i, ...,0], batch_speckle[i,...,0], data_range=1.0)
        gt_vs_pred  = structural_similarity(batch_clean[i, ...,0], batch_pred[i,...,0], data_range=1.0)
        print('   %i  \t\t  PSNR \t\t             %.2f             \t\t             %.2f      \t\t             %.2f' % (i, gt_vs_gt, gt_vs_in, gt_vs_pred))
        print('-------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
import sys
sys.path.append('speckle2void')
sys.path.append('SAR2SAR-GRD-test')
sys.path.append('CNN-NLM')
sys.path.insert(0, './libraries')

In [ ]:
handler = DatasetHandler('dataset')
print("Training dataset size: ", len(handler.train_paths))
print("Validation dataset size: ", len(handler.val_paths))
print("Testing dataset size: ", len(handler.test_paths))

In [ ]:
IMG_SHAPE = (96,96,1)

In [ ]:
batch_speckle, batch_clean = handler.data_loader_v2(
    paths      = handler.test_paths, 
    img_shape  = IMG_SHAPE)

# Speckle 2 void

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from Speckle2Void import Speckle2V

In [ ]:
tf.reset_default_graph()
batch_size=16

dir_train = None
dir_test = None

file_checkpoint = 'speckle2void/s2v_checkpoint/model.ckpt-299999'#None for the latest checkpoint

model = Speckle2V(dir_train,
                  dir_test,
                  file_checkpoint,
                  batch_size=batch_size,
                  patch_size=64,
                  model_name='speckle2void',
                  lr=1e-04, 
                  steps_per_epoch=2000,
                  k_penalty_tv=5e-05,
                  shift_list=[3,1],
                  prob = [0.9,0.1],
                  clip=1,
                  norm=1,
                  L_noise=1)   

In [ ]:
model.build_inference()
model.load_weights()

In [ ]:
batch_pred = model.predict(batch_speckle)

In [ ]:
plot_model_results(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], batch_pred[:,:64,:64,...])

In [ ]:
compute_metrics(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], batch_pred[:,:64,:64,...])

# SAR2SAR

In [ ]:
IMG_SHAPE = (256,256,1)

In [ ]:
batch_speckle, batch_clean = handler.data_loader_v2(
    paths      = handler.test_paths, 
    img_shape  = IMG_SHAPE)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from model import denoiser


with tf.Session() as sess:
    model = denoiser(sess)
    model.load('SAR2SAR-GRD-test/checkpoint')
    Y_ = tf.placeholder(tf.float32, [None, None, None, 1],
                                 name='clean_image')
    pred = sess.run([model.Y], feed_dict={model.Y_: batch_speckle})

In [ ]:
plot_model_results(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], pred[0][:,:64,:64,...])
compute_metrics(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], pred[0][:,:64,:64,...])

# CNN-NLM

In [ ]:
!pip install git+https://github.com/szagoruyko/pyinn.git@master

In [ ]:
sys.path.append('CNN-NLM')
sys.path.append('CNN-NLM/utils')

In [ ]:
import models.DnCNN as DnCNN
with open("CNN-NLM/weights/sar_sync/SAR_CNN_e50.pkl", "rb") as fid:
    dncnn_opt = dict(**pickle.load(fid).dncnn)
    dncnn_opt["residual"] = True
net = DnCNN.DnCNN(1, 1, **dncnn_opt)
net.load_state_dict(torch.load('CNN-NLM/weights/sar_sync/SAR_CNN_e50.t7', map_location=torch.device('cpu'))['net'])
pad = 0

def preprocessing_int2net(img):
    return img.abs().log()/2

def postprocessing_net2int(img):
    return (2*img).exp()

In [ ]:
out = np.zeros((batch_speckle.shape[0], 64, 64, 1))

for i in range(batch_speckle.shape[0]):
    noise_int  = np.float32((1/256)+batch_speckle[i,:64,:64,0])
    use_cuda = False

    with torch.no_grad():
        if use_cuda:
            net = net.cuda()

        noise_int  = torch.from_numpy(noise_int)[None, None, :, :]   
        noisy_pad = noise_int**2
        noisy_pad =  preprocessing_int2net(noisy_pad)

        if use_cuda:
            pred_int = net(noisy_pad.cuda()).cpu()
        else:
            pred_int = net(noisy_pad)

        pred_int = postprocessing_net2int(pred_int)
        pred_amp  = pred_int.abs().sqrt()



        out[i,...,0]   = pred_amp[0,0,::].numpy()
    

In [ ]:
plot_model_results(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], out)
compute_metrics(batch_speckle[:,:64,:64,...], batch_clean[:,:64,:64,...], out)